# 📊 HR Analytics – Analyse et Préparation des Données
## Projet TechNova Partners

Ce notebook présente l’ensemble du travail réalisé :
- Nettoyage et préparation des données RH (SIRH, évaluations, sondages).
- Analyse descriptive et exploration visuelle.
- Construction d’un DataFrame central.
- Tests de modélisation pour la prédiction du départ des employés.

🎯 **Objectif final :** fournir au CODIR des analyses fiables et actionnables sur la rétention et la gestion des talents.

In [ ]:
# Import des librairies principales
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

%matplotlib inline
sns.set(style="whitegrid")
pd.set_option('display.max_columns', None)

## 1️⃣ Chargement des données
Nous disposons de 3 fichiers principaux :
- **SIRH** : informations RH (âge, revenu, ancienneté, etc.)
- **Évaluations** : résultats des entretiens annuels
- **Sondages** : climat social et satisfaction des collaborateurs

In [ ]:
sirh = pd.read_csv('data/extrait_sirh.csv')
evals = pd.read_csv('data/extrait_eval.csv')
sondage = pd.read_csv('data/extrait_sondage.csv')

print('=== SIRH ===')
display(sirh.head())
print('=== Eval ===')
display(evals.head())
print('=== Sondage ===')
display(sondage.head())

## 2️⃣ Exploration initiale des données
Vérification de la structure des fichiers (types de variables, valeurs manquantes, distributions).

In [ ]:
print('--- Info SIRH ---')
sirh.info()
print('--- Info Eval ---')
evals.info()
print('--- Info Sondage ---')
sondage.info()

## 3️⃣ Nettoyage et harmonisation
- Normalisation des noms de colonnes
- Création d’un identifiant commun (`id_employee`)
- Alignement des trois fichiers

In [ ]:
sirh.columns = sirh.columns.str.lower().str.strip()
evals.columns = evals.columns.str.lower().str.strip()
sondage.columns = sondage.columns.str.lower().str.strip()

evals['id_employee'] = evals['eval_number'].str.replace('E_', '').astype(int)
sondage = sondage.rename(columns={'code_sondage': 'id_employee'})

## 4️⃣ Construction du DataFrame central
Jointure des différentes sources pour obtenir une base unique.

In [ ]:
df = pd.merge(sirh, evals, on='id_employee', how='inner')
df = pd.merge(df, sondage, on='id_employee', how='inner')
display(df.head())

## 5️⃣ Statistiques descriptives
Premières observations selon le statut de départ des employés.

In [ ]:
display(df.groupby('a_quitte_l_entreprise').mean(numeric_only=True))

## 6️⃣ Visualisations exploratoires
Exemple : relation entre le revenu, le poste et le départ des collaborateurs.

In [ ]:
plt.figure(figsize=(6,4))
sns.boxplot(x='a_quitte_l_entreprise', y='revenu_mensuel', data=df)
plt.title("Revenu mensuel vs Départ de l'entreprise")
plt.show()

plt.figure(figsize=(10,4))
sns.countplot(x='poste', hue='a_quitte_l_entreprise', data=df)
plt.title("Poste vs Départ de l'entreprise")
plt.xticks(rotation=45)
plt.show()

## 7️⃣ Préparation des données pour la modélisation
- Séparation features/target
- Encodage des variables catégorielles

In [ ]:
X = df.drop(columns=['a_quitte_l_entreprise', 'id_employee'])
y = df['a_quitte_l_entreprise']

def encode_categorical_features(df, ordinal_cols=None, nominal_cols=None):
    df_encoded = df.copy()
    if ordinal_cols:
        for col, categories in ordinal_cols.items():
            enc = OrdinalEncoder(categories=[categories])
            df_encoded[col] = enc.fit_transform(df[[col]])
    if nominal_cols:
        enc = OneHotEncoder(drop='first', sparse_output=False)
        nominal_encoded = enc.fit_transform(df[nominal_cols])
        nominal_encoded_df = pd.DataFrame(nominal_encoded, columns=enc.get_feature_names_out(nominal_cols), index=df.index)
        df_encoded = pd.concat([df_encoded.drop(columns=nominal_cols), nominal_encoded_df], axis=1)
    return df_encoded

ordinal_mapping = {'niveau_education': [1,2,3,4,5]}
nominal_cols = ['genre', 'departement', 'poste', 'domaine_etude', 'ayant_enfants', 'frequence_deplacement', 'heure_supplementaires']
X_encoded = encode_categorical_features(X, ordinal_cols=ordinal_mapping, nominal_cols=nominal_cols)
display(X_encoded.head())

## 8️⃣ Analyse des corrélations
Vérification des relations entre variables numériques et avec la cible.

In [ ]:
numeric_cols = X_encoded.select_dtypes(exclude=['object']).columns

plt.figure(figsize=(10,8))
sns.heatmap(X_encoded[numeric_cols].corr(), annot=False, cmap='coolwarm')
plt.title('Corrélations de Pearson')
plt.show()

## 9️⃣ Modélisation
Nous comparons trois modèles :
- **DummyClassifier** (baseline)
- **Régression Logistique**
- **RandomForestClassifier**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42, stratify=y)

models = {
    'Dummy': DummyClassifier(strategy='most_frequent'),
    'LogisticRegression': LogisticRegression(max_iter=1000),
    'RandomForest': RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')
}

for name, model in models.items():
    print(f'=== Modèle : {name} ===')
    model.fit(X_train, y_train)
    print('Test Metrics:')
    y_test_pred = model.predict(X_test)
    print(classification_report(y_test, y_test_pred, zero_division=0))

## 🔟 Évaluation avancée (RandomForest)
- Matrice de confusion
- Importance des variables

In [ ]:
best_model = models['RandomForest']
y_test_pred = best_model.predict(X_test)

# Matrice de confusion
ConfusionMatrixDisplay.from_estimator(best_model, X_test, y_test, cmap="Blues")
plt.title("Matrice de confusion - RandomForest")
plt.show()

# Importance des variables
importances = best_model.feature_importances_
indices = np.argsort(importances)[::-1][:10]

plt.figure(figsize=(8,5))
sns.barplot(x=importances[indices], y=X_train.columns[indices])
plt.title("Top 10 variables importantes (RandomForest)")
plt.show()